# Import the Libraries

In [169]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from statsmodels.stats import weightstats as ssw
from scipy.stats import chi2_contingency
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
from sklearn.metrics import classification_report

# Import Datasets

In [170]:
df = pd.read_csv('census_income.csv')
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,annual_income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


# Basic Information 

In [159]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education-num   32561 non-null  int64 
 5   marital-status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital-gain    32561 non-null  int64 
 11  capital-loss    32561 non-null  int64 
 12  hours-per-week  32561 non-null  int64 
 13  native-country  32561 non-null  object
 14  annual_income   32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


## Target Variable Information

In [160]:
df["annual_income"].unique()

array(['<=50K', '>50K'], dtype=object)

In [161]:
df.annual_income.value_counts()

annual_income
<=50K    24720
>50K      7841
Name: count, dtype: int64

In [171]:
df['annual_income'] = df['annual_income'].map({'<=50K': 0, '>50K': 1})
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,annual_income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,0
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,0
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,0
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,0
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,0


In [202]:
df = df.drop('fnlwgt', axis = 1)

## Convert values in binary format


In [163]:
df['annual_income'] = df['annual_income'].map({'<=50K': 0, '>50K': 1})
df.annual_income.value_counts()

Series([], Name: count, dtype: int64)

We labeled the values of
* '<=50K'     with     0
* '>50K'      with     1

## Replace missing placeholders '?' with 'others'1

In [164]:
for col in ['workclass', 'occupation', 'native-country']:
    df[col] = df[col].replace('?', 'others')

In [165]:
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,annual_income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,NaN
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,NaN
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,NaN
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,NaN
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,NaN


Here we labeled the values 
<=50K': 0
'>50K': 1
for proper interpretation and prediction

In [166]:
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,annual_income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,NaN
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,NaN
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,NaN
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,NaN
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,NaN


## Apply Categorical Features - Chi Square Test for Degree of Association

In [172]:
cat_cols = []
conti_cols = []
for i in df.columns:
  if df[i].dtype == 'object':
    print(i)
    print(np.unique(df[i]))
#----> print the column name and its unique values to manually inspect categories 

workclass
['?' 'Federal-gov' 'Local-gov' 'Never-worked' 'Private' 'Self-emp-inc'
 'Self-emp-not-inc' 'State-gov' 'Without-pay']
education
['10th' '11th' '12th' '1st-4th' '5th-6th' '7th-8th' '9th' 'Assoc-acdm'
 'Assoc-voc' 'Bachelors' 'Doctorate' 'HS-grad' 'Masters' 'Preschool'
 'Prof-school' 'Some-college']
marital-status
['Divorced' 'Married-AF-spouse' 'Married-civ-spouse'
 'Married-spouse-absent' 'Never-married' 'Separated' 'Widowed']
occupation
['?' 'Adm-clerical' 'Armed-Forces' 'Craft-repair' 'Exec-managerial'
 'Farming-fishing' 'Handlers-cleaners' 'Machine-op-inspct' 'Other-service'
 'Priv-house-serv' 'Prof-specialty' 'Protective-serv' 'Sales'
 'Tech-support' 'Transport-moving']
relationship
['Husband' 'Not-in-family' 'Other-relative' 'Own-child' 'Unmarried' 'Wife']
race
['Amer-Indian-Eskimo' 'Asian-Pac-Islander' 'Black' 'Other' 'White']
sex
['Female' 'Male']
native-country
['?' 'Cambodia' 'Canada' 'China' 'Columbia' 'Cuba' 'Dominican-Republic'
 'Ecuador' 'El-Salvador' 'England' '

* cat_cols = [] ---> to store the names of categorical columns
* conti_cols = [] ---> to store the names of Continuous columns

In [173]:
for i in df.columns:
  if df[i].dtype == 'object':
    print("-----------------------")
    print(f"Ho: {i} has no influence on annual_income")
    test_df = pd.crosstab(df[i], df.annual_income)
    chi2, p, *_ = chi2_contingency(test_df)
    if p > 0.05:
      print(p)
      print('We do not reject the Null hypothesis (Ho).')
    else:
      cat_cols.append(i)
      print(p)
      print('We reject the Null-hypothesis')

-----------------------
Ho: workclass has no influence on annual_income
2.0265054311207156e-220
We reject the Null-hypothesis
-----------------------
Ho: education has no influence on annual_income
0.0
We reject the Null-hypothesis
-----------------------
Ho: marital-status has no influence on annual_income
0.0
We reject the Null-hypothesis
-----------------------
Ho: occupation has no influence on annual_income
0.0
We reject the Null-hypothesis
-----------------------
Ho: relationship has no influence on annual_income
0.0
We reject the Null-hypothesis
-----------------------
Ho: race has no influence on annual_income
2.305960610160958e-70
We reject the Null-hypothesis
-----------------------
Ho: sex has no influence on annual_income
0.0
We reject the Null-hypothesis
-----------------------
Ho: native-country has no influence on annual_income
2.2113858852543023e-44
We reject the Null-hypothesis


- This code performs a Chi-Square test of independence between each categorical column and the target variable annual_income.
- To identify which categorical features are statistically associated with the income class (<=50K or >50K), and keep only those relevant features.
- The Chi-Square test is used to find dependency/association between two categorical variables. Since both the feature (i) and the target (annual_income) are categorical, this is the correct test to use.

## Analyze the distribution of categories within the 'workclass' and 'occupation' columns.

In [188]:
test_workclass = df.workclass.value_counts(normalize=True).reset_index()
test_occupation = df.occupation.value_counts(normalize=True).reset_index()
test_native = df['native-country'].value_counts(normalize=True).reset_index()

print("*****************************************************************************************")
print(test_workclass)
print("*****************************************************************************************")
print(test_occupation)
print("*****************************************************************************************")
print(test_native)
print("*****************************************************************************************")

*****************************************************************************************
          workclass  proportion
0           Private    0.697030
1  Self-emp-not-inc    0.078038
2         Local-gov    0.064279
3            others    0.056386
4         State-gov    0.039864
5      Self-emp-inc    0.034274
6       Federal-gov    0.029483
7       Without-pay    0.000430
8      Never-worked    0.000215
*****************************************************************************************
           occupation  proportion
0      Prof-specialty    0.127146
1        Craft-repair    0.125887
2     Exec-managerial    0.124873
3        Adm-clerical    0.115783
4               Sales    0.112097
5       Other-service    0.101195
6   Machine-op-inspct    0.061485
7              others    0.056601
8    Transport-moving    0.049046
9   Handlers-cleaners    0.042075
10    Farming-fishing    0.030527
11       Tech-support    0.028500
12    Protective-serv    0.019932
13    Priv-house-serv   

- To know the different proportions of categories in the column workclass and occupation

## Above column contains missing values '?' lets replace it with 'others'

In [189]:
for i in ['workclass', 'occupation', 'native-country']:
  df[i] = df[i].replace('?', 'others')

In [190]:
print("*****************************************************************************************")
print(df['workclass'].unique())
print("*****************************************************************************************")
print(df['occupation'].unique())
print("*****************************************************************************************")
print(df['native-country'].unique())
print("*****************************************************************************************")

*****************************************************************************************
['State-gov' 'Self-emp-not-inc' 'Private' 'Federal-gov' 'Local-gov'
 'others' 'Self-emp-inc' 'Without-pay' 'Never-worked']
*****************************************************************************************
['Adm-clerical' 'Exec-managerial' 'Handlers-cleaners' 'Prof-specialty'
 'Other-service' 'Sales' 'Craft-repair' 'Transport-moving'
 'Farming-fishing' 'Machine-op-inspct' 'Tech-support' 'others'
 'Protective-serv' 'Armed-Forces' 'Priv-house-serv']
*****************************************************************************************
['United-States' 'Cuba' 'Jamaica' 'India' 'others' 'Mexico' 'South'
 'Puerto-Rico' 'Honduras' 'England' 'Canada' 'Germany' 'Iran'
 'Philippines' 'Italy' 'Poland' 'Columbia' 'Cambodia' 'Thailand' 'Ecuador'
 'Laos' 'Taiwan' 'Haiti' 'Portugal' 'Dominican-Republic' 'El-Salvador'
 'France' 'Guatemala' 'China' 'Japan' 'Yugoslavia' 'Peru'
 'Outlying-US(Guam-USVI-etc

In [191]:
for i in df.columns:
  if df[i].dtype != 'object' and i != 'annual_income':
    print(i)

age
fnlwgt
education-num
capital-gain
capital-loss
hours-per-week


- Lists all continuous (numeric) columns other than the target column annual_income.

In [192]:
for i in df.columns:
    if df[i].dtype != 'object' and i != 'annual_income':
        print("*****************************************************************************************")
        print(f"{i} has no influence on annual_income")
        t, p = ssw.ztest(df[df.annual_income == 1][i], df[df.annual_income == 0][i], value=0)
        print(p)
        if p > 0.05:
            print('We do not reject the null hypothesis (Ho).')
        else:
            conti_cols.append(i)
            print("We reject the null hypothesis (Ho)")


*****************************************************************************************
age has no influence on annual_income
0.0
We reject the null hypothesis (Ho)
*****************************************************************************************
fnlwgt has no influence on annual_income
0.08772712748728925
We do not reject the null hypothesis (Ho).
*****************************************************************************************
education-num has no influence on annual_income
0.0
We reject the null hypothesis (Ho)
*****************************************************************************************
capital-gain has no influence on annual_income
0.0
We reject the null hypothesis (Ho)
*****************************************************************************************
capital-loss has no influence on annual_income
3.5734849240534076e-166
We reject the null hypothesis (Ho)
*****************************************************************************************


- Use the Two-sample Z-test to compare the mean values of feature i between:
- People with income >50K (annual_income == 1)
- People with income <=50K (annual_income == 0)

- To check if the feature significantly differs between the two income groups.
- If it does, then it’s a good predictor of income and should be kept for modeling.

In [195]:
final_cols = cat_cols + conti_cols
final_cols
# Columns who influence annual Income

['workclass',
 'education',
 'marital-status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'native-country',
 'age',
 'education-num',
 'capital-gain',
 'capital-loss',
 'hours-per-week',
 'age',
 'education-num',
 'capital-gain',
 'capital-loss',
 'hours-per-week']

In [196]:
test_dummy = pd.DataFrame() #--------> Initializes an empty DataFrame to store all the dummy-encoded columns.

In [199]:
test_dummy = pd.DataFrame()

for i in cat_cols:
    dummy = pd.get_dummies(df[i], drop_first=True, prefix=f"{i}").astype(int)
    test_dummy = pd.concat([test_dummy, dummy], axis=1)

    print(f"\n✅ One-hot encoding completed for: '{i}'")
    print(f"Unique categories in original column: {df[i].unique()}")
    print(f"Generated dummy columns: {list(dummy.columns)}")

print("\n✅ All categorical features successfully encoded.")



✅ One-hot encoding completed for: 'workclass'
Unique categories in original column: ['State-gov' 'Self-emp-not-inc' 'Private' 'Federal-gov' 'Local-gov'
 'others' 'Self-emp-inc' 'Without-pay' 'Never-worked']
Generated dummy columns: ['workclass_Local-gov', 'workclass_Never-worked', 'workclass_Private', 'workclass_Self-emp-inc', 'workclass_Self-emp-not-inc', 'workclass_State-gov', 'workclass_Without-pay', 'workclass_others']

✅ One-hot encoding completed for: 'education'
Unique categories in original column: ['Bachelors' 'HS-grad' '11th' 'Masters' '9th' 'Some-college' 'Assoc-acdm'
 'Assoc-voc' '7th-8th' 'Doctorate' 'Prof-school' '5th-6th' '10th'
 '1st-4th' 'Preschool' '12th']
Generated dummy columns: ['education_11th', 'education_12th', 'education_1st-4th', 'education_5th-6th', 'education_7th-8th', 'education_9th', 'education_Assoc-acdm', 'education_Assoc-voc', 'education_Bachelors', 'education_Doctorate', 'education_HS-grad', 'education_Masters', 'education_Preschool', 'education_Prof-

- This block performs one-hot encoding on the selected categorical columns, which are identified as relevant through statistical testing. 
- It generates binary (0/1) indicator variables for each category (excluding the first to prevent multicollinearity), and stores the results in a combined DataFrame (test_dummy). 
- This encoded data is then prepared for merging with the original dataset for use in model training.

In [200]:
test_dummy.columns


Index(['workclass_Local-gov', 'workclass_Never-worked', 'workclass_Private',
       'workclass_Self-emp-inc', 'workclass_Self-emp-not-inc',
       'workclass_State-gov', 'workclass_Without-pay', 'workclass_others',
       'education_11th', 'education_12th', 'education_1st-4th',
       'education_5th-6th', 'education_7th-8th', 'education_9th',
       'education_Assoc-acdm', 'education_Assoc-voc', 'education_Bachelors',
       'education_Doctorate', 'education_HS-grad', 'education_Masters',
       'education_Preschool', 'education_Prof-school',
       'education_Some-college', 'marital-status_Married-AF-spouse',
       'marital-status_Married-civ-spouse',
       'marital-status_Married-spouse-absent', 'marital-status_Never-married',
       'marital-status_Separated', 'marital-status_Widowed',
       'occupation_Armed-Forces', 'occupation_Craft-repair',
       'occupation_Exec-managerial', 'occupation_Farming-fishing',
       'occupation_Handlers-cleaners', 'occupation_Machine-op-inspct',

- Above displays the column names of your encoded (one-hot) features, created from the selected categorical columns (cat_cols).

In [201]:
df = df.drop(columns=cat_cols)
df = pd.concat([df, test_dummy], axis=1)
df.columns

Index(['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss',
       'hours-per-week', 'annual_income', 'workclass_Local-gov',
       'workclass_Never-worked', 'workclass_Private',
       ...
       'native-country_Puerto-Rico', 'native-country_Scotland',
       'native-country_South', 'native-country_Taiwan',
       'native-country_Thailand', 'native-country_Trinadad&Tobago',
       'native-country_United-States', 'native-country_Vietnam',
       'native-country_Yugoslavia', 'native-country_others'],
      dtype='object', length=101)

- This block finalizes the transformation of categorical features into numerical format using one-hot encoding, integrates them into the main dataset, and removes the original categorical variables to avoid redundancy.
- The resulting DataFrame df is fully numeric and ready for model training.

In [204]:
df_preds = df.drop('annual_income', axis = 1)
df_resp = df['annual_income']

- Split predictors and response columns

In [205]:
df_preds.shape

(32561, 99)

In [206]:
x_train, x_test, y_train, y_test = train_test_split(df_preds, df_resp, test_size=0.2)
vif_data = pd.DataFrame()
vif_data['Columns'] = conti_cols
test_df = x_train[conti_cols]
res = []
for ctr in range(test_df.shape[1]):
  res.append(variance_inflation_factor(test_df.values, ctr))
vif_data['VIF'] = res
vif_data

C:\Python-3.10\lib\site-packages\statsmodels\stats\outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


,Columns,VIF
0,age,2.206920e+09
1,education-num,2.147054e+09
2,capital-gain,inf
3,capital-loss,inf
4,hours-per-week,inf
5,age,inf
6,education-num,inf
7,capital-gain,inf
8,capital-loss,3.277420e+09
9,hours-per-week,1.224178e+07


- This block checks your continuous variables for multicollinearity using VIF. 
- Features with a high VIF may need to be removed to improve model stability and interpretability.

In [207]:
mod = sm.Logit(y_train, x_train).fit()
print(mod.summary())

         Current function value: 0.314651
         Iterations: 35
                           Logit Regression Results                           
Dep. Variable:          annual_income   No. Observations:                26048
Model:                          Logit   Df Residuals:                    25950
Method:                           MLE   Df Model:                           97
Date:                Mon, 16 Jun 2025   Pseudo R-squ.:                  0.4291
Time:                        00:13:40   Log-Likelihood:                -8196.0
converged:                      False   LL-Null:                       -14357.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                                coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------
age                                           0.0262      0.002     14.210      0.

C:\Python-3.10\lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


- This line trains the logistic regression model and provides a detailed statistical report on how each feature affects income classification, enabling both prediction and explainability

In [208]:
y_pred = mod.predict(x_test)

Performing the prediction step of your logistic regression model using the test data

In [209]:
y_pred_class = (y_pred >= 0.5).astype(int)
print(classification_report(y_test, y_pred_class))

              precision    recall  f1-score   support

           0       0.88      0.93      0.90      4926
           1       0.73      0.60      0.66      1587

    accuracy                           0.85      6513
   macro avg       0.80      0.77      0.78      6513
weighted avg       0.84      0.85      0.84      6513



This block converts predicted probabilities into binary class labels, then evaluates the model’s performance using precision, recall, f1-score, and accuracy.

### **Classification Report Summary**

* **Overall Accuracy:** 85%

* **Class 0 (≤50K income):**

  * **Precision:** 0.88 → 88% of predicted low-income cases were correct.
  * **Recall:** 0.93 → 93% of actual low-income individuals were correctly identified.
  * **F1-score:** 0.90 → Excellent balance between precision and recall.

* **Class 1 (>50K income):**

  * **Precision:** 0.73 → 73% of predicted high-income cases were correct.
  * **Recall:** 0.60 → Only 60% of actual high-income individuals were detected.
  * **F1-score:** 0.66 → Moderate performance; room for improvement.

- The model is **very good at detecting low-income individuals**, but **misses a significant portion of high-income ones**. Consider tuning the threshold or using class balancing techniques to improve recall for class 1.
